In [1]:
# Update Data/ directory.
%run ./DownloadData.py

In [2]:
import os
import pandas as pd

In [3]:
df = {}
for data in os.listdir('Data/'):
    df[data[:-4]] = pd.read_csv('Data/' + data)
print sorted(df.keys())

['caste', 'communicabledisease1', 'communicabledisease2', 'hdi2011', 'mentalhealth', 'poverty1', 'poverty2', 'sexratio', 'vaccinations', 'watersanitaion']
